In [1]:
from confluent_kafka import Producer
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import io
import json
import base64
from glob import glob
from PIL import Image
import random
import time

In [2]:
# Get all audio paths for specific machine type
fans = glob('/opt/audio_files/*_fan/*/*/*.wav')
random.shuffle(fans)

In [3]:
# Kafka Configuration
KAFKA_BROKER = "kafka:9092"
TOPIC = "audio_spectrograms"

# Kafka Producer
producer = Producer({"bootstrap.servers": KAFKA_BROKER})

In [4]:
def create_spectrogram(audio_path):
    # Load the .wav file
    audio_vector, sr = librosa.load(audio_path, sr=None)

    # Set the figure size and DPI to get a 224x224 image
    fig_size = 224 / 100  # inches (224 pixels / 100 DPI)
    dpi = 100  # Dots per inch

    # Create Mel spectrogram
    fig, ax = plt.subplots(figsize=(fig_size, fig_size), dpi=dpi)
    ax.set_axis_off()
    fig.subplots_adjust(left=0, right=1, top=1, bottom=0)  # Remove padding

    sgram = librosa.stft(audio_vector)  # Extract STFT
    sgram_mag, _ = librosa.magphase(sgram)
    mel_scale_sgram = librosa.feature.melspectrogram(S=sgram_mag, sr=sr)
    mel_sgram = librosa.amplitude_to_db(mel_scale_sgram, ref=np.min)  # Convert to dB

    librosa.display.specshow(mel_sgram, sr=sr, ax=ax)

    # Save spectrogram as PNG in memory buffer
    buf = io.BytesIO()
    plt.savefig(buf, format='png', bbox_inches='tight', pad_inches=0)
    plt.close()

    buf.seek(0)
    spectrogram_bytes = buf.getvalue()  # Get raw PNG bytes
    
    return spectrogram_bytes

In [5]:
for audio_path in fans:
    file_name = audio_path.split("/")[-1]
    audio_class = audio_path.split("/")[-2]  # Extract class from folder structure
    spectrogram_bytes = create_spectrogram(audio_path)

    # Create Kafka message (Metadata)
    metadata = json.dumps({"file_name": file_name, "class": audio_class}).encode("utf-8")
    
    # Send message header (Metadata) + Raw PNG image
    producer.produce(TOPIC, key=file_name, value=spectrogram_bytes, headers=[("metadata", metadata)])
    producer.flush()

    print(f"Produced Spectrogram: {file_name}, Class {audio_class}")

    # Wait 1 second before sending the next message
    time.sleep(1)

Produced Spectrogram: 00000677.wav, Class normal
Produced Spectrogram: 00000322.wav, Class abnormal
Produced Spectrogram: 00000631.wav, Class normal
Produced Spectrogram: 00000342.wav, Class normal
Produced Spectrogram: 00000855.wav, Class normal
Produced Spectrogram: 00000515.wav, Class normal
Produced Spectrogram: 00000795.wav, Class normal
Produced Spectrogram: 00000166.wav, Class normal
Produced Spectrogram: 00000857.wav, Class normal
Produced Spectrogram: 00000235.wav, Class normal
Produced Spectrogram: 00000876.wav, Class normal
Produced Spectrogram: 00000412.wav, Class normal
Produced Spectrogram: 00000724.wav, Class normal
Produced Spectrogram: 00000863.wav, Class normal
Produced Spectrogram: 00000485.wav, Class normal
Produced Spectrogram: 00000554.wav, Class normal
Produced Spectrogram: 00000325.wav, Class normal
Produced Spectrogram: 00000915.wav, Class normal
Produced Spectrogram: 00000856.wav, Class normal
Produced Spectrogram: 00000390.wav, Class normal
Produced Spectrogr

KeyboardInterrupt: 